In [2]:
using BenchmarkTools, Random

## We have $\texttt{randperms}$, which is a length $t$ vector of length $n$ random permutations. 

In [3]:
n = 100
t = 100
randperms = [randperm(n) for i in 1:t];

## Task : Do $\texttt{patiencesort.(randperms)}$ fast

In [4]:
function patiencesort1(p)
    # p : Permutation
    # Returns length of longest increasing subsequence
    pile_tops = Int[]
    for α ∈ p       
        whichpile = 1+sum(α.>pile_tops) # first pile where α is smaller
        if  whichpile ≤ length(pile_tops)
            pile_tops[whichpile] = α   # put α on top of a pile  or ..
        else
            push!(pile_tops, α)        # create a new pile
        end
    end
    return length(pile_tops)
end

patiencesort1 (generic function with 1 method)

In [5]:
@btime patiencesort1.(randperms);

  639.959 μs (20304 allocations: 985.31 KiB)


## $\texttt{whichpile = 1+sum(α.>pile_tops)}$ creates unnecessary allocations. 

## Let us use pre-defined function $\texttt{searchsortedfirst}$ for that

In [6]:
function patiencesort2(p)
    # p : Permutation
    # Returns length of longest increasing subsequence
    pile_tops = Int[]
    for α ∈ p       
        whichpile = searchsortedfirst(pile_tops, α) # first pile where α is smaller
        if  whichpile ≤ length(pile_tops)
            pile_tops[whichpile] = α   # put α on top of a pile  or ..
        else
            push!(pile_tops, α)        # create a new pile
        end
    end
    return length(pile_tops)
end

patiencesort2 (generic function with 1 method)

In [7]:
@btime patiencesort2.(randperms);

  129.750 μs (304 allocations: 47.81 KiB)


## How about we make our own $\texttt{seachsortedfirst}$ ?

In [8]:
function my_searchsortedfirst(pt, α)
   for i in 1:length(pt) #length(pt)
        if α < pt[i] return i end
    end
    return length(pt)+1
end    
function patiencesort3(p)
    # p : Permutation
    # Returns length of longest increasing subsequence
    pile_tops = Int[]
    for α ∈ p       
        whichpile = my_searchsortedfirst(pile_tops, α) # first pile where α is smaller
        if  whichpile ≤ length(pile_tops)
            pile_tops[whichpile] = α   # put α on top of a pile  or ..
        else
            push!(pile_tops, α)        # create a new pile
        end
    end
    return length(pile_tops)
end

patiencesort3 (generic function with 1 method)

In [9]:
@btime patiencesort3.(randperms);

  112.916 μs (304 allocations: 47.81 KiB)


## Also pushing values into the vector is not efficient. Let us preallocate $\texttt{pile_tops}$ vector. From math, let us assume that we are given that the length of the longest increasing subsequence longer than $3\sqrt{n}$ happens with less than probability $10^{-10}$. 

## Since $\texttt{length(pile_tops)}$ will not give as the real length, we would need modify $\texttt{searchsortedfirst}$ too. 

In [10]:
function my_searchsortedfirst(pt, α, lpt)
    for i in 1:lpt #length(pt)
        if α < pt[i] return i end
    end
    return lpt+1
end    
function patiencesort4(p)
    # p : Permutation
    # Returns length of longest increasing subsequence
    pile_tops = zeros(Int, round(Int, 3*sqrt(length(p))))
    lengthpt = 0
    for α ∈ p       
        whichpile = my_searchsortedfirst(pile_tops, α, lengthpt) # first pile where α is smaller
        if  whichpile ≤ lengthpt
            pile_tops[whichpile] = α   # put α on top of a pile  or ..
        else
            pile_tops[lengthpt+1] = α  
            lengthpt += 1        
        end
    end
    return lengthpt
end

patiencesort4 (generic function with 1 method)

In [11]:
@btime patiencesort4.(randperms);

  101.208 μs (104 allocations: 30.62 KiB)


### See that allocation number went down from 300 to 100

## If you feel you are safe with indices, you can use $\texttt{@inbounds}$.

In [12]:
function my_searchsortedfirst(pt, α, lpt)
    @inbounds for i in 1:lpt #length(pt)
         if α < pt[i] return i end
    end
    return lpt+1
end    
function patiencesort5(p)
    # p : Permutation
    # Returns length of longest increasing subsequence
    pile_tops = zeros(Int, round(Int, 3*sqrt(length(p))))
    lengthpt = 0
    @inbounds for α ∈ p       
        whichpile = my_searchsortedfirst(pile_tops, α, lengthpt) # first pile where α is smaller
        if  whichpile ≤ lengthpt
            pile_tops[whichpile] = α   # put α on top of a pile  or ..
        else
            pile_tops[lengthpt+1] = α  
            lengthpt += 1        
        end
    end
    return lengthpt
end

patiencesort5 (generic function with 1 method)

In [13]:
@btime patiencesort5.(randperms);

  86.542 μs (104 allocations: 30.62 KiB)


# From here you can even add some parallelism, @threads, pmap, GPU, ...